In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform





In [2]:

data = pd.read_csv('modified_data7.csv', encoding='cp1252')

# 'rating' 열에 10을 곱하여 타겟을 자연수로 변환
data['rating'] *= 10

# 필요없는 피처 날리기
data = data.drop("subbedOutTime", axis=1)
data = data.drop("position", axis=1)
data = data.drop("fantasyScore", axis=1)

# 포지션만 있고 세부 포지션 널값처리
# 'positionStringShort'의 널값이 있는 행 찾기
null_positions = data[data['positionStringShort'].isna()]

# 각 널값에 대해 처리
for index, row in null_positions.iterrows():
    # 현재 널값이 있는 행의 'name' 가져오기
    name = row['name']
    # 해당 'name'을 가진 모든 행에서 'positionStringShort'의 최빈값 찾기
    most_frequent_position = data[data['name'] == name]['positionStringShort'].mode()[0]
    # 널값 채우기
    data.at[index, 'positionStringShort'] = most_frequent_position

data = data.drop("name", axis=1)
# rating 널값 제거, 개수가 매우적어 없는거만 제거
data = data.dropna(subset=["rating"])

# x , y 나누기
data_x = data.drop("rating", axis=1)
data_y = data["rating"]

# 학습, 테스트 데이터 나누기
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)

# 'positionStringShort'와 'positionRow' 열 선택
x_train_cat = x_train[['positionStringShort', 'positionRow']].values
# OneHotEncoder 객체 생성
cat_encoder = OneHotEncoder()
# 원-핫 인코딩 적용
x_train_cat_1hot = cat_encoder.fit_transform(x_train_cat)
# positionStringShort, positionRow은 카테고리컬
x_train_num = x_train.drop(["positionStringShort", "positionRow"], axis=1)

# 불리언 타입이 아닌 수치형 데이터만 선택
numeric_features = x_train_num.select_dtypes(include=['int64', 'float64'])
# StandardScaler 객체 생성
scaler = StandardScaler()
# 수치형 데이터에 표준화 적용
numeric_features_scaled = scaler.fit_transform(numeric_features)
# 스케일링된 데이터를 데이터프레임으로 변환 (원본 데이터프레임의 인덱스 사용)
numeric_features_scaled_df = pd.DataFrame(numeric_features_scaled, columns=numeric_features.columns,
                                          index=x_train_num.index)
# 데이터를 소수점 셋째 자리에서 반올림
numeric_features_scaled_df = numeric_features_scaled_df.round(3)
# 스케일링된 수치형 데이터프레임을 원본 데이터프레임과 병합
data_num_scaled = x_train_num.copy()
data_num_scaled[numeric_features.columns] = numeric_features_scaled_df

# 원-핫 인코딩된 데이터를 데이터프레임으로 변환
x_train_cat_1hot_df = pd.DataFrame(x_train_cat_1hot.toarray(), columns=cat_encoder.get_feature_names_out(),
                                   index=x_train.index)

# 불리언 타입의 데이터만 선택
bool_features = x_train_num.select_dtypes(include=['bool'])


# 스케일링된 수치형 데이터, 불리언 타입의 데이터, 원-핫 인코딩된 데이터 병합
x_train_prepared = pd.concat([data_num_scaled, bool_features, x_train_cat_1hot_df], axis=1)


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lin_reg = LinearRegression()
scores = cross_val_score(lin_reg, x_train_prepared, y_train, scoring= "neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

In [4]:
print(rmse_scores)

[5.39361414 4.93581561 5.16028153 5.45502356 5.24959012 4.85149871
 4.86613414 5.1388391  5.27589023 4.96707092]


In [5]:
print(rmse_scores.mean())

5.129375804704698


In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# cross_val_score가 완료된 후에는 학습된 모델이 저장되지 않습니다

lin_reg2 = LinearRegression()
lin_reg2.fit(x_train_prepared, y_train)

prediction = lin_reg2.predict(x_train_prepared)
lin_mse = mean_squared_error(y_train, prediction)
lin_rmse = np.sqrt(lin_mse)
lin_rmse



5.105424969786548

In [7]:
# MAE 계산
mae = mean_absolute_error(y_train, prediction)
mae

3.5899727079538803

In [8]:
x_test, y_test


# 'positionStringShort'와 'positionRow' 열 선택
x_test_cat = x_test[['positionStringShort', 'positionRow']].values
# OneHotEncoder 객체 생성
cat_encoder2 = OneHotEncoder()
# 원-핫 인코딩 적용
x_test_cat_1hot = cat_encoder2.fit_transform(x_test_cat)

# positionStringShort, positionRow은 카테고리컬
x_test_num = x_test.drop(["positionStringShort", "positionRow"], axis=1)

# 불리언 타입이 아닌 수치형 데이터만 선택
numeric_features2 = x_test_num.select_dtypes(include=['int64', 'float64'])
# StandardScaler 객체 생성
scaler2 = StandardScaler()
# 수치형 데이터에 표준화 적용
numeric_features_scaled2 = scaler.transform(numeric_features2)
# 스케일링된 데이터를 데이터프레임으로 변환 (원본 데이터프레임의 인덱스 사용)
numeric_features_scaled_df2 = pd.DataFrame(numeric_features_scaled2, columns=numeric_features2.columns,
                                          index=x_test_num.index)
# 데이터를 소수점 셋째 자리에서 반올림
numeric_features_scaled_df2 = numeric_features_scaled_df2.round(3)
# 스케일링된 수치형 데이터프레임을 원본 데이터프레임과 병합
data_num_scaled2 = x_test_num.copy()
data_num_scaled2[numeric_features2.columns] = numeric_features_scaled_df2

# 원-핫 인코딩된 데이터를 데이터프레임으로 변환
x_test_cat_1hot_df = pd.DataFrame(x_test_cat_1hot.toarray(), columns=cat_encoder2.get_feature_names_out(),
                                   index=x_test.index)

# 불리언 타입의 데이터만 선택
bool_features2 = x_test_num.select_dtypes(include=['bool'])


# 스케일링된 수치형 데이터, 불리언 타입의 데이터, 원-핫 인코딩된 데이터 병합
x_test_prepared = pd.concat([data_num_scaled2, bool_features2, x_test_cat_1hot_df], axis=1)

In [9]:
prediction2 = lin_reg2.predict(x_test_prepared)
lin_mse = mean_squared_error(y_test, prediction2)
lin_rmse = np.sqrt(lin_mse)
lin_rmse
#아니 이렇게 다 작아버리면 어떡하지? 다른 모델도 해보자

5.591957290467666

In [10]:
# 테스트 데이터에 대한 예측을 생성합니다
y_pred = lin_reg2.predict(x_test_prepared)

# MAE 계산
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)


Mean Absolute Error: 3.91806282202957


In [11]:
print(x_train_prepared.columns)
print(x_train_prepared.shape)

Index(['isCaptain', 'minutesPlayed', 'isGoalkeeper', 'goal', 'assist',
       'yellowCard', 'redCard', 'total_shots', 'accurate_passes_rate(%)',
       'chances_created', 'expected_goals(xG)',
       'expected_goals_on_target(xGOT)', 'expected_assists(xA)', 'xG+xA',
       'shot_accuracy(%)', 'blocked_shots', 'touches',
       'touches_in_opposition_box', 'passes_into_final_third',
       'long_balls_accuracy(%)', 'dispossessed', 'xG_Non_penalty',
       'tackles_won_rate(%)', 'clearances', 'headed_clearance',
       'defensive_actions', 'recoveries', 'dribbled_past', 'duels_won',
       'duels_lost', 'ground_duels_won_rate(%)', 'aerial_duels_won_rate(%)',
       'was_fouled', 'fouls_commited', 'dribble_success_rate(%)',
       'penalties_won', 'big_chances_missed', 'crosses_success_rate(%)',
       'isCaptain', 'isGoalkeeper', 'x0_AM', 'x0_CB', 'x0_CM', 'x0_DM',
       'x0_GK', 'x0_LB', 'x0_LM', 'x0_LW', 'x0_RB', 'x0_RM', 'x0_RW', 'x0_ST',
       'x1_0', 'x1_1', 'x1_2', 'x1_3', 'x1_4'

In [12]:
eta = 0.1  # 학습률
n_iterations = 1000  # 반복 횟수
m = len(x_train_prepared)  # 샘플 개수

# x_train_prepared와 y_train을 NumPy 배열로 변환
X_b = np.c_[np.ones((m, 1)), x_train_prepared.values]  # x0 = 1 추가
y = y_train.values.reshape(-1, 1)

theta = np.random.randn(X_b.shape[1], 1)  # theta 초기화

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

In [13]:
# 테스트 데이터 준비 (1을 추가해야 합니다)
X_test_b = np.c_[np.ones((len(x_test_prepared), 1)), x_test_prepared.values]

# 테스트 데이터에 대한 예측 수행
y_pred = X_test_b.dot(theta)

test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)

print("Test MSE:", test_mse)
print("Test RMSE:", test_rmse)

Test MSE: 31.280835238342505
Test RMSE: 5.592927251300745


In [14]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=5000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(x_train_prepared, y_train)

# 테스트 데이터에 대한 예측을 생성합니다
prediction3 = sgd_reg.predict(x_test_prepared)

sgd_mse = mean_squared_error(y_test, prediction3)
sgd_rmse = np.sqrt(sgd_mse)
print("Root Mean Squared Error:", sgd_rmse)
# MAE 계산
sgd_mae = mean_absolute_error(y_test, prediction3)
print("Mean Absolute Error:", sgd_mae)


Root Mean Squared Error: 71716578302.2314
Mean Absolute Error: 5725556127.485703


In [15]:
# L1 규제를 적용한 SGDRegressor 모델 생성
sgd_reg_l1 = SGDRegressor(max_iter=1000, tol=1e-3, penalty='l1', eta0=0.01)
sgd_reg_l1.fit(x_train_prepared, y_train)

# 테스트 데이터에 대한 예측 생성 및 성능 평가
prediction_l1 = sgd_reg_l1.predict(x_test_prepared)
mse_l1 = mean_squared_error(y_test, prediction_l1)
rmse_l1 = np.sqrt(mse_l1)
print("L1 규제 RMSE:", rmse_l1)
mae_l1 = mean_absolute_error(y_test, prediction_l1)
print("L1 규제 MAE:", mae_l1)
print(prediction_l1[0])
print(y_test.iloc[0])

L1 규제 RMSE: 5.601449375106179
L1 규제 MAE: 3.9346237679150944
65.55577668593455
68.0


In [16]:
# L2 규제를 적용한 SGDRegressor 모델 생성
sgd_reg_l2 = SGDRegressor(max_iter=1000, tol=1e-3, penalty='l2', eta0=0.01)
sgd_reg_l2.fit(x_train_prepared, y_train)

# 테스트 데이터에 대한 예측 생성 및 성능 평가
prediction_l2 = sgd_reg_l2.predict(x_test_prepared)
mse_l2 = mean_squared_error(y_test, prediction_l2)
rmse_l2 = np.sqrt(mse_l2)
print("L2 규제 RMSE:", rmse_l2)
mae_l2 = mean_absolute_error(y_test, prediction_l2)
print("L2 규제 MAE:", mae_l1)
print(prediction_l2[0])
print(y_test.iloc[0])

L2 규제 RMSE: 5.611720782681402
L2 규제 MAE: 3.9346237679150944
65.44101953360965
68.0


In [17]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(x_train_prepared, y_train)

# 테스트 데이터에 대한 예측 생성 및 성능 평가
prediction_ela = sgd_reg_l2.predict(x_test_prepared)
mse_ela = mean_squared_error(y_test, prediction_ela)
rmse_ela = np.sqrt(mse_ela)
print("elastic 규제 RMSE:", rmse_ela)
mae_ela = mean_absolute_error(y_test, prediction_ela)
print("elastic 규제 MAE:", mae_l1)

elastic 규제 RMSE: 5.611720782681402
elastic 규제 MAE: 3.9346237679150944


In [18]:
# SGDRegressor의 하이퍼파라미터 범위 설정
param_distribs = {
    'alpha': uniform(0.0001, 0.01),  # 규제 파라미터
    'penalty': ['l2', 'l1', 'elasticnet', 'None'],  # 규제 종류
    'eta0': uniform(0.01, 0.1)  # 학습률
}

# RandomizedSearchCV 설정
sgd_reg = SGDRegressor(max_iter=5000, tol=1e-3)
rnd_search = RandomizedSearchCV(sgd_reg, param_distributions=param_distribs,
                                n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42, return_train_score=True)

# 랜덤 탐색 실행
rnd_search.fit(x_train_prepared, y_train)

# 결과 평가
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

# 최적의 파라미터
print("최적의 파라미터:", rnd_search.best_params_)

67849707324.97589 {'alpha': 0.0038454011884736248, 'eta0': 0.10507143064099161, 'penalty': 'elasticnet'}
50291183298.15336 {'alpha': 0.007896910002727693, 'eta0': 0.0696850157946487, 'penalty': 'l1'}
5.1593269913649875 {'alpha': 0.0016599452033620266, 'eta0': 0.015808361216819947, 'penalty': 'None'}
5.152963744327461 {'alpha': 0.0034370861113902185, 'eta0': 0.02428668179219408, 'penalty': 'elasticnet'}
87100418384.2463 {'alpha': 0.0003058449429580245, 'eta0': 0.10699098521619943, 'penalty': 'None'}
5.145852987357484 {'alpha': 0.009485527090157502, 'eta0': 0.010077876584101433, 'penalty': 'None'}
5.185594139091758 {'alpha': 0.0019340450985343381, 'eta0': 0.04042422429595378, 'penalty': 'l1'}
5.148148086283851 {'alpha': 0.00017066305219717406, 'eta0': 0.012306242504141576, 'penalty': 'elasticnet'}
5.171778407429555 {'alpha': 0.006218528947223795, 'eta0': 0.023949386065204185, 'penalty': 'None'}
5.188227698133592 {'alpha': 0.00983755518841459, 'eta0': 0.033277134043030426, 'penalty': 'l1'

In [19]:
# 최적의 하이퍼파라미터로 SGDRegressor 모델 초기화
optimal_sgd_reg = SGDRegressor(alpha=0.001752669390630025, eta0=0.011563640674119394, penalty='l2', 
                               max_iter=1000, tol=1e-3)

# 모델 학습
optimal_sgd_reg.fit(x_train_prepared, y_train)

# 테스트 데이터에 대한 예측
y_pred_optimal = optimal_sgd_reg.predict(x_test_prepared)

# 성능 평가: MSE와 RMSE
optimal_mse = mean_squared_error(y_test, y_pred_optimal)
optimal_rmse = np.sqrt(optimal_mse)
print("Optimal RMSE:", optimal_rmse)

# 성능 평가: MAE
optimal_mae = mean_absolute_error(y_test, y_pred_optimal)
print("Optimal MAE:", optimal_mae)

Optimal RMSE: 5.59178038641786
Optimal MAE: 3.922524925236262


In [20]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)

# 수치형 데이터에 대한 다항 특성 추가


x_train_num_poly = poly_features.fit_transform(data_num_scaled)
# 변환된 수치형 데이터를 데이터프레임으로 변환 (원본 데이터프레임의 인덱스 사용)
x_train_num_poly_df = pd.DataFrame(x_train_num_poly, index=x_train_num.index)
# 스케일링된 수치형 데이터, 불리언 타입의 데이터, 원-핫 인코딩된 데이터 병합
x_train_prepared_poly = pd.concat([x_train_num_poly_df, bool_features, x_train_cat_1hot_df], axis=1)

print(x_train_prepared.shape)
print(x_train_prepared_poly.shape)



x_test_num_poly = poly_features.transform(data_num_scaled2)
# 변환된 수치형 데이터를 데이터프레임으로 변환 (원본 데이터프레임의 인덱스 사용)
x_test_num_poly_df = pd.DataFrame(x_test_num_poly, index=x_test_num.index)
# 스케일링된 수치형 데이터, 불리언 타입의 데이터, 원-핫 인코딩된 데이터 병합
x_test_prepared_poly = pd.concat([x_test_num_poly_df, bool_features2, x_test_cat_1hot_df], axis=1)




(8686, 58)
(8686, 799)


In [21]:
#  Polynomial Regression 모델 생성

sgd_reg_poly = SGDRegressor(max_iter=10000, tol=1e-3, penalty='l1', eta0=0.01)
sgd_reg_poly.fit(x_train_prepared_poly, y_train)

# 테스트 데이터에 대한 예측
prediction_sgd_reg_poly = sgd_reg_poly.predict(x_test_prepared_poly)

# 성능 평가: MSE와 RMSE
sgd_reg_poly_mse = mean_squared_error(y_test, prediction_sgd_reg_poly)
sgd_reg_poly_rmse = np.sqrt(sgd_reg_poly_mse)
print("Optimal RMSE:", sgd_reg_poly_rmse)

# 성능 평가: MAE
sgd_reg_poly_mae = mean_absolute_error(y_test, prediction_sgd_reg_poly)
print("Optimal MAE:", sgd_reg_poly_mae)

print(prediction_sgd_reg_poly[0])
print(y_test.iloc[0])

C:\Users\junhe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\junhe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Optimal RMSE: 2388010431921.1997
Optimal MAE: 391982779591.7112
-28229751748.028835
68.0


In [22]:
#  Polynomial Regression 모델 생성
lin_reg_poly = LinearRegression()
lin_reg_poly.fit(x_train_prepared_poly, y_train)

# 테스트 데이터에 대한 예측
prediction_lin_reg_poly = lin_reg_poly.predict(x_test_prepared_poly)

# 성능 평가: MSE와 RMSE
lin_reg_poly_mse = mean_squared_error(y_test, prediction_lin_reg_poly)
lin_reg_poly_rmse = np.sqrt(lin_reg_poly_mse)
print("RMSE:", lin_reg_poly_rmse)

# 성능 평가: MAE
lin_reg_poly_mae = mean_absolute_error(y_test, prediction_lin_reg_poly)
print("MAE:", lin_reg_poly_mae)

# 예측값과 실제값 출력
print("Predicted:", prediction_lin_reg_poly[15])
print("Actual:", y_test.iloc[15])


# 결과는 좋은데 -> 왜 rmse, mae가 이따위지?
# svm 커널 ->2차와 관련성??


C:\Users\junhe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\junhe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RMSE: 418594944467.9428
MAE: 15502572843.912329
Predicted: 66.7647705078125
Actual: 68.0
